In [1]:
cat > Dockerfile <<EOF
FROM ubuntu:18.04
RUN apt-get update && apt-get install -y software-properties-common && add-apt-repository -y ppa:alex-p/tesseract-ocr
RUN apt-get update && apt-get install -y tesseract-ocr-all 
RUN mkdir /work
WORKDIR /work
EOF

In [2]:
sudo apt-get install -y dtrx rename

Reading package lists... Done
Building dependency tree       
Reading state information... Done
dtrx is already the newest version (7.1-1).
rename is already the newest version (0.20-7).
The following packages were automatically installed and are no longer required:
  gyp libjs-async libjs-inherits libjs-node-uuid libuv1-dev node-abbrev
  node-ansi node-ansi-color-table node-archy node-async node-balanced-match
  node-block-stream node-brace-expansion node-combined-stream node-concat-map
  node-cookie-jar node-delayed-stream node-forever-agent node-form-data
  node-fs.realpath node-fstream node-fstream-ignore node-github-url-from-git
  node-glob node-graceful-fs node-hosted-git-info node-inflight node-inherits
  node-ini node-isexe node-json-stringify-safe node-lockfile node-lru-cache
  node-mime node-minimatch node-mkdirp node-mute-stream node-node-uuid
  node-nopt node-npmlog node-once node-osenv node-path-is-absolute
  node-pseudomap node-qs node-read node-request node-retry node-ri

In [3]:
docker build -t tess - < Dockerfile

Sending build context to Docker daemon  2.048kB
Step 1/5 : FROM ubuntu:18.04
 ---> 1d9c17228a9e
Step 2/5 : RUN apt-get update && apt-get install -y software-properties-common && add-apt-repository -y ppa:alex-p/tesseract-ocr
 ---> Using cache
 ---> 8b47ec2893a2
Step 3/5 : RUN apt-get update && apt-get install -y tesseract-ocr-all
 ---> Using cache
 ---> 214800589b8e
Step 4/5 : RUN mkdir /work
 ---> Using cache
 ---> fb3a8b6e8b8c
Step 5/5 : WORKDIR /work
 ---> Using cache
 ---> ff11cee2dca5
Successfully built ff11cee2dca5
Successfully tagged tess:latest


In [4]:
cat > run-tess <<\EOF
#!/bin/bash
docker run -v $(pwd):/work -w /work -t tess tesseract "$@"
EOF
chmod 755 run-tess

In [ ]:
run-tess --help

Usage:
  tesseract --help | --help-extra | --version
  tesseract --list-langs
  tesseract imagename outputbase [options...] [configfile...]

OCR options:
  -l LANG[+LANG]        Specify language(s) used for OCR.
NOTE: These options must occur before any configfile.

Single options:
  --help                Show this help message.
  --help-extra          Show extra help for advanced users.
  --version             Show version information.
  --list-langs          List available languages for tesseract engine.


In [ ]:
gsutil ls gs://lpr-g1000/ | head

gs://lpr-g1000/Volume_0000.tgz
gs://lpr-g1000/Volume_0001.tgz
gs://lpr-g1000/Volume_0002.tgz
gs://lpr-g1000/Volume_0003.tgz
gs://lpr-g1000/Volume_0004.tgz
gs://lpr-g1000/Volume_0005.tgz
gs://lpr-g1000/Volume_0006.tgz
gs://lpr-g1000/Volume_0007.tgz
gs://lpr-g1000/Volume_0008.tgz
gs://lpr-g1000/Volume_0009.tgz


In [ ]:
gsutil cat gs://lpr-g1000/Volume_0000.tgz | tar -ztvf - | head

drwxrwxr-x tmb/tmb           0 2017-11-20 12:50 Volume_0000/
-rw-rw-rw- tmb/tmb      196530 2007-06-27 18:43 Volume_0000/Image_0000.JPEG
-rw-rw-r-- tmb/tmb         265 2017-11-20 12:47 Volume_0000/Image_0000.hocr
-rw-rw-rw- tmb/tmb      138388 2007-06-27 18:43 Volume_0000/Image_0001.JPEG
-rw-rw-r-- tmb/tmb       19977 2017-11-15 15:54 Volume_0000/Image_0001.bin.png
-rw-rw-r-- tmb/tmb         476 2017-11-20 12:47 Volume_0000/Image_0001.hocr
-rw-rw-r-- tmb/tmb       17774 2017-11-20 12:48 Volume_0000/Image_0001.lines.png
-rw-rw-r-- tmb/tmb      619993 2017-11-15 15:54 Volume_0000/Image_0001.nrm.png
-rw-rw-r-- tmb/tmb       18569 2017-11-20 12:47 Volume_0000/Image_0001.pseg.png
-rw-rw-rw- tmb/tmb       85894 2007-06-27 18:43 Volume_0000/Image_0002.JPEG
tar: write error


In [ ]:
for vol in {0000..0999}; do
cd $HOME/ocr/runtess || exit 1
gsutil cp gs://lpr-g1000/Volume_$vol.tgz volume.tgz
rm -rf volume outputs
dtrx --one rename volume.tgz
mkdir outputs
cp volume/*.JPEG outputs/.
ls outputs/*.JPEG | wc -l
cd $HOME/ocr/runtess/outputs || exit 1
test -f Image_0000.JPEG || exit 1
for fname in Image_????.JPEG; do
   echo Volume_$vol/$fname
   base=$(basename $fname .JPEG)
   date >> LOG
   ../run-tess --oem 1 $fname $base -l eng hocr >> LOG
done
cd $HOME/ocr/runtess || exit 1
test -d outputs || exit 1
mv outputs Volume_$vol &&
tar --sort=name -zcf Volume_$vol.tgz Volume_$vol &&
gsutil cp Volume_$vol.tgz gs://lpr-g1000-tess/Volume_$vol_tessjpg.tgz
rm -rf volume outputs Volume_$vol Volume_$vol.tgz
done

Copying gs://lpr-g1000/Volume_0000.tgz...
\ [1 files][749.5 MiB/749.5 MiB]                                                
Operation completed over 1 objects/749.5 MiB.                                    
518
Volume_0000/Image_0000.JPEG
Volume_0000/Image_0001.JPEG
Volume_0000/Image_0002.JPEG
Volume_0000/Image_0003.JPEG
Volume_0000/Image_0004.JPEG
Volume_0000/Image_0005.JPEG
Volume_0000/Image_0006.JPEG
Volume_0000/Image_0007.JPEG
Volume_0000/Image_0008.JPEG
Volume_0000/Image_0009.JPEG
Volume_0000/Image_0010.JPEG
Volume_0000/Image_0011.JPEG
Volume_0000/Image_0012.JPEG
Volume_0000/Image_0013.JPEG
Volume_0000/Image_0014.JPEG
Volume_0000/Image_0015.JPEG
Volume_0000/Image_0016.JPEG
Volume_0000/Image_0017.JPEG
Volume_0000/Image_0018.JPEG
Volume_0000/Image_0019.JPEG
Volume_0000/Image_0020.JPEG
Volume_0000/Image_0021.JPEG
Volume_0000/Image_0022.JPEG
Volume_0000/Image_0023.JPEG
Volume_0000/Image_0024.JPEG
Volume_0000/Image_0025.JPEG
Volume_0000/Image_0026.JPEG
Volume_0000/Image_0027.JPEG
Volume_

In [ ]:
for vol in {0000..0999}; do
cd $HOME/ocr/runtess || exit 1
gsutil cp gs://lpr-g1000/Volume_$vol.tgz volume.tgz
rm -rf volume outputs
dtrx --one rename volume.tgz
mkdir outputs
cp volume/*.nrm.png outputs/.
ls outputs/*.nrm.png | wc -l
cd $HOME/ocr/runtess/outputs || exit 1
for fname in Image_????.nrm.png; do
   echo Volume_$vol/$fname
   base=$(basename $fname .nrm.png)
   date >> LOG
   ../run-tess --oem 1 $fname $base -l eng hocr >> LOG
done
cd $HOME/ocr/runtess || exit 1
test -d outputs || exit 1
mv outputs Volume_$vol &&
tar --sort=name -zcf Volume_$vol.tgz Volume_$vol &&
gsutil cp Volume_$vol.tgz gs://lpr-g1000-tess/Volume_$vol_tessnrm.tgz
rm -rf volume outputs Volume_$vol Volume_$vol.tgz
done